In [1]:
import pandas as pd
from pandas import DataFrame,Series
from datetime import datetime
from functools import reduce
import numpy as np
import json

In [2]:
data = pd.ExcelFile("jay_lyrics.xlsx")
data.sheet_names

['专辑']

In [3]:
df = data.parse("专辑")
df.head()

,发布日期,所属专辑,歌曲名,歌词,作词人
0,2000-11-01,JAY,可爱女人,想要有直升机 想要和你飞到宇宙去\n想要和你融化在一起 融化在银河里\n我每天每天每天在想想...,徐若瑄
1,2000-11-01,JAY,完美主义,如果说怀疑 可以造句 如果说分离 能够翻译\n如果这一切 真的可以 我想要将我的寂寞封闭\n...,方文山
2,2000-11-01,JAY,星晴,一步两步三步四步 望著天 手牵手\n一颗两颗三颗四颗 连成线 看星星\n一步两步三步四步 望...,周杰伦
3,2000-11-01,JAY,娘子,娘子 娘子却依旧每日 折一枝杨柳\n你在那里 在小村外的溪边河口 默默等著我\n娘子依旧每日...,方文山
4,2000-11-01,JAY,斗牛,有什么不妥 有话就直说\n别窝在角落 不爽就反驳\n到底跩什么 懂不懂篮球\n有种不要走 三...,方文山


# 缺失值检查

In [4]:
cols = list(df.columns)
missing = list(map(lambda x:df[x].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'作词人': 0, '发布日期': 0, '所属专辑': 0, '歌曲名': 0, '歌词': 0}

# 发布日期

In [5]:
df['发布日期'].dtypes

dtype('<M8[ns]')

In [6]:
df['发布日期'].describe()

count                     150
unique                     14
top       2005-11-01 00:00:00
freq                       12
first     2000-11-01 00:00:00
last      2016-06-24 00:00:00
Name: 发布日期, dtype: object

# 专辑

In [7]:
df['所属专辑'].value_counts()

哎呦，不错哦      12
十二新作        12
十一月的萧邦      12
魔杰座         11
叶惠美         11
惊叹号         11
跨时代         11
周杰伦的床边故事    10
JAY         10
八度空间        10
我很忙         10
FANTASY     10
依然范特西       10
七里香         10
Name: 所属专辑, dtype: int64

# 歌曲名

In [8]:
df['歌曲名'].unique()

array(['可爱女人', '完美主义', '星晴', '娘子', '斗牛', '黑色幽默', '伊斯坦堡', '印地安老斑鸠', '龙卷风',
       '反方向的钟', '爱在西元前', '爸我回来了', '简单爱', '忍者', '开不了口', '上海一九四三', '对不起',
       '威廉古堡', '双截棍', '安静', '半兽人', '半岛铁盒', '暗号', '龙拳', '火车叨位去', '分裂',
       '爷爷泡的茶', '回到过去', '米兰的小铁匠', '最后的战役', '以父之名', '懦夫', '晴天', '三年二班',
       '东风破', '你听得到', '同一种调调', '她的睫毛', '爱情悬崖', '梯田', '双刀', '我的地盘', '七里香',
       '借口', '外婆', '将军', '搁浅', '乱舞春秋', '困兽之斗', '园游会', '止战之殇', '夜曲', '蓝色风暴',
       '发如雪', '黑色毛衣', '四面楚歌', '枫', '浪漫手机', '逆鳞', '麦芽糖', '珊瑚海', '漂移',
       '一路向北', '夜的第七章', '听妈妈的话', '千里之外', '本草纲目', '退后', '红模仿', '心雨', '白色风车',
       '迷迭香', '菊花台', '牛仔很忙', '彩虹', '青花瓷', '阳光宅男', '蒲公英的约定', '无双', '我不配',
       '扯', '甜甜的', '最长的电影', '龙战骑士', '给我一首歌的时间', '蛇舞', '花海', '魔术先生',
       '说好的幸福呢', '兰亭序', '流浪诗人', '时光机', '乔克叔叔', '稻香', '跨时代', '说了再见', '烟花易冷',
       '免费教学录影带', '好久不见', '雨下一整晚', '嘻哈空姐', '我落泪?情绪零碎', '爱的飞行日记', '自导自演',
       '超人不会飞', '惊叹号', '迷魂曲', 'Mine Mine', '公主病', '你好吗', '疗伤烧肉粽', '琴伤',
       '水手怕水', '世界未末日', '皮影戏', '超跑女神', '四季列车', '手语', '公共

# 作词人

In [9]:
df['作词人'].unique()

array(['徐若瑄', '方文山', '周杰伦', '许世昌', '刘畊宏', '黄俊郎', '曾郁婷', '宋健彰', '\xa0周杰伦',
       '古小力/黄凌嘉', '罗宇轩/李汪哲', '唐从圣', '黄凌嘉', '林义杰', '罗宇轩/黄婕熙'], dtype=object)

# 歌词

## 歌词分句

In [10]:
data = df.copy()

In [11]:
data.head()

,发布日期,所属专辑,歌曲名,歌词,作词人
0,2000-11-01,JAY,可爱女人,想要有直升机 想要和你飞到宇宙去\n想要和你融化在一起 融化在银河里\n我每天每天每天在想想...,徐若瑄
1,2000-11-01,JAY,完美主义,如果说怀疑 可以造句 如果说分离 能够翻译\n如果这一切 真的可以 我想要将我的寂寞封闭\n...,方文山
2,2000-11-01,JAY,星晴,一步两步三步四步 望著天 手牵手\n一颗两颗三颗四颗 连成线 看星星\n一步两步三步四步 望...,周杰伦
3,2000-11-01,JAY,娘子,娘子 娘子却依旧每日 折一枝杨柳\n你在那里 在小村外的溪边河口 默默等著我\n娘子依旧每日...,方文山
4,2000-11-01,JAY,斗牛,有什么不妥 有话就直说\n别窝在角落 不爽就反驳\n到底跩什么 懂不懂篮球\n有种不要走 三...,方文山


In [12]:
data1 = data.apply(lambda x: {'歌曲名':x['歌曲名'],'发布日期': x['发布日期'],'所属专辑':x['所属专辑'],'作词人':x['作词人'],'歌词':x['歌词'].split('\n')}, axis = 1)
data1.head()

,发布日期,所属专辑,歌曲名,歌词,作词人
0,2000-11-01,JAY,可爱女人,"[想要有直升机 想要和你飞到宇宙去, 想要和你融化在一起 融化在银河里, 我每天每天每天在想...",徐若瑄
1,2000-11-01,JAY,完美主义,"[如果说怀疑 可以造句 如果说分离 能够翻译, 如果这一切 真的可以 我想要将我的寂寞封闭,...",方文山
2,2000-11-01,JAY,星晴,"[一步两步三步四步 望著天 手牵手, 一颗两颗三颗四颗 连成线 看星星, 一步两步三步四步 ...",周杰伦
3,2000-11-01,JAY,娘子,"[娘子 娘子却依旧每日 折一枝杨柳, 你在那里 在小村外的溪边河口 默默等著我, 娘子依旧每...",方文山
4,2000-11-01,JAY,斗牛,"[有什么不妥 有话就直说, 别窝在角落 不爽就反驳, 到底跩什么 懂不懂篮球, 有种不要走 ...",方文山


In [21]:
series1 = data1['歌词']
series1.head()

0    [想要有直升机 想要和你飞到宇宙去, 想要和你融化在一起 融化在银河里, 我每天每天每天在想...
1    [如果说怀疑 可以造句 如果说分离 能够翻译, 如果这一切 真的可以 我想要将我的寂寞封闭,...
2    [一步两步三步四步 望著天 手牵手, 一颗两颗三颗四颗 连成线 看星星, 一步两步三步四步 ...
3    [娘子 娘子却依旧每日 折一枝杨柳, 你在那里 在小村外的溪边河口 默默等著我, 娘子依旧每...
4    [有什么不妥 有话就直说, 别窝在角落 不爽就反驳, 到底跩什么 懂不懂篮球, 有种不要走 ...
Name: 歌词, dtype: object

In [20]:
data1.sub(series1)

C:\Users\Kismet\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:3033: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


TypeError: 

In [13]:
dat = data1[['发布日期','所属专辑','歌曲名','作词人']].copy()
dat.head()

,发布日期,所属专辑,歌曲名,作词人
0,2000-11-01,JAY,可爱女人,徐若瑄
1,2000-11-01,JAY,完美主义,方文山
2,2000-11-01,JAY,星晴,周杰伦
3,2000-11-01,JAY,娘子,方文山
4,2000-11-01,JAY,斗牛,方文山


In [28]:
for i,j in data1['歌词'].apply(list).iteritems():
    print (i,j)

0 ['想要有直升机 想要和你飞到宇宙去', '想要和你融化在一起 融化在银河里', '我每天每天每天在想想想想着你', '这样的甜蜜 让我开始相信命运', '感谢地心引力 让我碰到你', '漂亮的让我面红的可爱女人', '温柔的让我心疼的可爱女人', '透明的让我感动的可爱女人', '坏坏的让我疯狂的可爱女人', '世界这样大而我而我只是只小小小的蚂蚁', '但我要尽全力全力全力保护你']
1 ['如果说怀疑 可以造句 如果说分离 能够翻译', '如果这一切 真的可以 我想要将我的寂寞封闭', '然后在这里 不限日期 然后将过去 慢慢温习', '让我爱上你 那场悲剧 是你完美演出的一场戏', '宁愿心碎哭泣 再狠狠忘记', '你爱过我的证据', '让晶莹的泪滴 闪烁成回忆', '伤人的美丽', '你的完美主义太彻底', '让我连恨都难以下笔', '将真心抽离写成日记', '像是一场默剧', '你的完美主义太彻底', '分手的话像语言暴力', '我已我无能为力再提起', '决定中断熟悉', '']
2 ['一步两步三步四步 望著天 手牵手', '一颗两颗三颗四颗 连成线 看星星', '一步两步三步四步 望著天 手牵手', '一颗两颗三颗四颗 连成线 看星', '乘著风 游荡在蓝天边 一片云掉落在我面前', '捏成你的形状 随风跟著我 一口一口吃掉忧愁', '载著你 彷佛载著阳光 不管到哪里都是晴天', '蝴蝶自在飞 花也布满天 一朵一朵因你而香', '试图让夕阳飞翔 带领你我环绕大自然', '迎著风 开始共渡每一天', '手牵手 一步两步三步四步 望著天', '看星星 一颗两颗三颗四颗 连成线', '背著背默默许下心愿', '看远方的星否听的见', '看远方的星如果听的见', '它一定实现', '']
3 ['娘子 娘子却依旧每日 折一枝杨柳', '你在那里 在小村外的溪边河口 默默等著我', '娘子依旧每日折一枝杨柳', '你在那里 在小村外的溪边 默默等着 娘子', '一壶好酒 再来一碗热粥 配上几斤的牛肉', '我说店小二 三两银够不够', '景色入秋 漫天黄沙凉过', '塞北的客栈人多 牧草有没有 我马儿有些瘦', '世事看透 江湖上潮起潮落 什么恩怨过错', '在多年以后 还是让人难过 心伤透', '娘子她人在江南等我 泪不休 语沉默', '娘子却依旧每日

In [36]:
b_flat = pd.DataFrame([[i,x] for i,y in data1['歌词'].apply(list).iteritems() for x in y],columns=list('IB'))
b_flat.head()

,I,B
0,0,想要有直升机 想要和你飞到宇宙去
1,0,想要和你融化在一起 融化在银河里
2,0,我每天每天每天在想想想想着你
3,0,这样的甜蜜 让我开始相信命运
4,0,感谢地心引力 让我碰到你


In [37]:
b_flat = b_flat.set_index("I")
b_flat.head()

,B
I,
0,想要有直升机 想要和你飞到宇宙去
0,想要和你融化在一起 融化在银河里
0,我每天每天每天在想想想想着你
0,这样的甜蜜 让我开始相信命运
0,感谢地心引力 让我碰到你


In [38]:
df = dat.merge(b_flat,left_index=True,right_index=True)
df.rename(columns={'B':'歌词'},inplace = True)

In [39]:
cols = list(df.columns)
missing = list(map(lambda x:df[x].isnull().sum(),cols))
missing_dict = dict(zip(cols,missing))
missing_dict

{'作词人': 0, '发布日期': 0, '所属专辑': 0, '歌曲名': 0, '歌词': 0}

In [40]:
cond = df['歌词']==""
df = df.loc[~cond]

In [41]:
df.index = range(len(df))
df.shape

(5771, 5)

In [42]:
df.head(30)

,发布日期,所属专辑,歌曲名,作词人,歌词
0,2000-11-01,JAY,可爱女人,徐若瑄,想要有直升机 想要和你飞到宇宙去
1,2000-11-01,JAY,可爱女人,徐若瑄,想要和你融化在一起 融化在银河里
2,2000-11-01,JAY,可爱女人,徐若瑄,我每天每天每天在想想想想着你
3,2000-11-01,JAY,可爱女人,徐若瑄,这样的甜蜜 让我开始相信命运
4,2000-11-01,JAY,可爱女人,徐若瑄,感谢地心引力 让我碰到你
5,2000-11-01,JAY,可爱女人,徐若瑄,漂亮的让我面红的可爱女人
6,2000-11-01,JAY,可爱女人,徐若瑄,温柔的让我心疼的可爱女人
7,2000-11-01,JAY,可爱女人,徐若瑄,透明的让我感动的可爱女人
8,2000-11-01,JAY,可爱女人,徐若瑄,坏坏的让我疯狂的可爱女人
9,2000-11-01,JAY,可爱女人,徐若瑄,世界这样大而我而我只是只小小小的蚂蚁


## 每首歌的关键词

In [43]:
import operator
import jieba
import re

In [44]:
to_remove = ['，', '。', '、', '：', '…', '  ', '   ', '=', '“', '”', '【', '】', '\n', "、", "。", "〈", "〉", "《", "》","回复","允悲","cry","知道", "一",
                 "一切", "一则", "一方面", "一旦", "一来", "一样", "一般", "七", "万一", "三", "上下", "不仅", "不但", "不光", "不单", "不只", "不如",
                 "不怕", "不惟", "不成", "不拘", "不比", "不然", "不特", "不独", "不管", "不论", "不过", "不问", "与", "与其", "与否", "与此同时", "且",
                 "两者", "个", "临", "为", "为了", "为什么", "为何", "为着", "乃", "乃至", "么", "之", "之一", "之所以", "之类", "乌乎", "乎", "乘",
                 "九", "也", "也好", "也罢", "了", "二", "于", "于是", "于是乎", "云云", "五", "人家", "什么", "什么样", "从", "从而", "他", "他人",
                 "他们", "以", "以便", "以免", "以及", "以至", "以至于", "以致", "们", "任", "任何", "任凭", "似的", "但", "但是", "何", "何况", "何处",
                 "何时", "作为", "你", "你们", "使得", "例如", "依", "依照", "俺", "俺们", "倘", "倘使", "倘或", "倘然", "倘若", "借", "假使", "假如",
                 "假若", "像", "八", "六", "兮", "关于", "其", "其一", "其中", "其二", "其他", "其余", "其它", "其次", "具体地说", "具体说来", "再者",
                 "再说", "冒", "冲", "况且", "几", "几时", "凭", "凭借", "则", "别", "别的", "别说", "到", "前后", "前者", "加之", "即", "即令",
                 "即使", "即便", "即或", "即若", "又", "及", "及其", "及至", "反之", "反过来", "反过来说", "另", "另一方面", "另外", "只是", "只有", "只要",
                 "只限", "叫", "叮咚", "可", "可以", "可是", "可见", "各", "各个", "各位", "各种", "各自", "同", "同时", "向", "向着", "吓", "吗",
                 "否则", "吧", "吧哒", "吱", "呀", "呃", "呕", "呗", "呜", "呜呼", "呢", "呵", "呸", "呼哧", "咋", "和", "咚", "咦", "咱",
                 "咱们", "咳", "哇", "哈", "哈哈", "哉", "哎", "哎呀", "哎哟", "哗", "哟", "哦", "哩", "哪", "哪个", "哪些", "哪儿", "哪天", "哪年",
                 "哪怕", "哪样", "哪边", "哪里", "哼", "哼唷", "唉", "啊", "啐", "啥", "啦", "啪达", "喂", "喏", "喔唷", "嗡嗡", "嗬", "嗯", "嗳",
                 "嘎", "嘎登", "嘘", "嘛", "嘻", "嘿", "四", "因", "因为", "因此", "因而", "固然", "在", "在下", "地", "多", "多少", "她", "她们",
                 "如", "如上所述", "如何", "如其", "如果", "如此", "如若", "宁", "宁可", "宁愿", "宁肯", "它", "它们", "对", "对于", "将", "尔后",
                 "尚且", "就", "就是", "就是说", "尽", "尽管", "岂但", "己", "并", "并且", "开外", "开始", "归", "当", "当着", "彼", "彼此", "往",
                 "待", "得", "怎", "怎么", "怎么办", "怎么样", "怎样", "总之", "总的来看", "总的来说", "总的说来", "总而言之", "恰恰相反", "您", "慢说", "我",
                 "我们", "或", "或是", "或者", "所", "所以", "打", "把", "抑或", "拿", "按", "按照", "换句话说", "换言之", "据", "接着", "故", "故此",
                 "旁人", "无宁", "无论", "既", "既是", "既然", "时候", "是", "是的", "替", "有", "有些", "有关", "有的", "望", "朝", "朝着", "本",
                 "本着", "来", "来着", "极了", "果然", "果真", "某", "某个", "某些", "根据", "正如", "此", "此外", "此间", "毋宁", "每", "每当", "比",
                 "比如", "比方", "沿", "沿着", "漫说", "焉", "然则", "然后", "然而", "照", "照着", "甚么", "甚而", "甚至", "用", "由", "由于",
                 "由此可见", "的", "的话", "相对而言", "省得", "着", "着呢", "矣", "离", "第", "等", "等等", "管", "紧接着", "纵", "纵令", "纵使",
                 "纵然", "经", "经过", "结果", "给", "继而", "综上所述", "罢了", "者", "而", "而且", "而况", "而外", "而已", "而是", "而言", "能", "腾",
                 "自", "自个儿", "自从", "自各儿", "自家", "自己", "自身", "至", "至于", "若", "若是", "若非", "莫若", "虽", "虽则", "虽然", "虽说",
                 "被", "要", "要不", "要不是", "要不然", "要么", "要是", "让", "论", "设使", "设若", "该", "诸位", "谁", "谁知", "赶", "起", "起见",
                 "趁", "趁着", "越是", "跟", "较", "较之", "边", "过", "还是", "还有", "这", "这个", "这么", "这么些", "这么样", "这么点儿", "这些",
                 "这会儿", "这儿", "这就是说", "这时", "这样", "这边", "这里", "进而", "连", "连同", "通过", "遵照", "那", "那个", "那么", "那么些",
                 "那么样", "那些", "那会儿", "那儿", "那时", "那样", "那边", "那里", "鄙人", "鉴于", "阿", "除", "除了", "除此之外", "除非", "随", "随着",
                 "零", "非但", "非徒", "靠", "顺", "顺着", "首先", "︿", "！", "＃", "＄", "％", "＆", "（", "）", "＊", "＋", "，", "０", "１",
                 "２", "３", "４", "５", "６", "７", "８", "９", "：", "；", "＜", "＞", "？", "＠", "［", "］", "｛", "｜", "｝", "～",
                 "￥"]

In [45]:
data.head()

,发布日期,所属专辑,歌曲名,歌词,作词人
0,2000-11-01,JAY,可爱女人,想要有直升机 想要和你飞到宇宙去\n想要和你融化在一起 融化在银河里\n我每天每天每天在想想...,徐若瑄
1,2000-11-01,JAY,完美主义,如果说怀疑 可以造句 如果说分离 能够翻译\n如果这一切 真的可以 我想要将我的寂寞封闭\n...,方文山
2,2000-11-01,JAY,星晴,一步两步三步四步 望著天 手牵手\n一颗两颗三颗四颗 连成线 看星星\n一步两步三步四步 望...,周杰伦
3,2000-11-01,JAY,娘子,娘子 娘子却依旧每日 折一枝杨柳\n你在那里 在小村外的溪边河口 默默等著我\n娘子依旧每日...,方文山
4,2000-11-01,JAY,斗牛,有什么不妥 有话就直说\n别窝在角落 不爽就反驳\n到底跩什么 懂不懂篮球\n有种不要走 三...,方文山


In [46]:
data.shape

(150, 5)

In [47]:
data['歌词'] = data['歌词'].str.replace(r"\s+"," ")
data['歌词'] = data['歌词'].str.replace(r"\n"," ")
data['歌词'].head()

0    想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在银河里 我每天每天每天在想想想想...
1    如果说怀疑 可以造句 如果说分离 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后...
2    一步两步三步四步 望著天 手牵手 一颗两颗三颗四颗 连成线 看星星 一步两步三步四步 望著天...
3    娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口 默默等著我 娘子依旧每日折一...
4    有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底跩什么 懂不懂篮球 有种不要走 三对三斗...
Name: 歌词, dtype: object

In [48]:
#提取关键词的函数
def get_key(line):
    words_dict = {}
    pattern = re.compile(r'[^一-龥]')
    line = re.sub(pattern,"",line)
    cut = jieba.cut(line)
    words = " ".join(cut).split(" ")
    for w in words:
        if w=="" or len(w)==1 or w in to_remove:
            continue
        elif w in words_dict:
            words_dict[w] +=1
        else:
            words_dict[w] = 1
    sorted_dict = sorted(words_dict.items(),key=operator.itemgetter(1))
    sorted_dict.reverse()

    return sorted_dict[0][0]+" "+sorted_dict[1][0]

In [49]:
data['歌词'][0]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在银河里 我每天每天每天在想想想想着你 这样的甜蜜 让我开始相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 世界这样大而我而我只是只小小小的蚂蚁 但我要尽全力全力全力保护你'

In [50]:
get_key(data['歌词'][0])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kismet\AppData\Local\Temp\jieba.cache
Loading model cost 1.017 seconds.
Prefix dict has been built succesfully.


'女人 可爱'

In [51]:
data['关键词'] = None
data.head()

,发布日期,所属专辑,歌曲名,歌词,作词人,关键词
0,2000-11-01,JAY,可爱女人,想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在银河里 我每天每天每天在想想想想...,徐若瑄,None
1,2000-11-01,JAY,完美主义,如果说怀疑 可以造句 如果说分离 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后...,方文山,None
2,2000-11-01,JAY,星晴,一步两步三步四步 望著天 手牵手 一颗两颗三颗四颗 连成线 看星星 一步两步三步四步 望著天...,周杰伦,None
3,2000-11-01,JAY,娘子,娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口 默默等著我 娘子依旧每日折一...,方文山,None
4,2000-11-01,JAY,斗牛,有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底跩什么 懂不懂篮球 有种不要走 三对三斗...,方文山,None


In [52]:
for i in range(len(data)):
    data['关键词'][i] = get_key(data['歌词'][i])

C:\Users\Kismet\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [53]:
data

,发布日期,所属专辑,歌曲名,歌词,作词人,关键词
0,2000-11-01,JAY,可爱女人,想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在银河里 我每天每天每天在想想想想...,徐若瑄,女人 可爱
1,2000-11-01,JAY,完美主义,如果说怀疑 可以造句 如果说分离 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后...,方文山,彻底 完美主义
2,2000-11-01,JAY,星晴,一步两步三步四步 望著天 手牵手 一颗两颗三颗四颗 连成线 看星星 一步两步三步四步 望著天...,周杰伦,连成线 四颗
3,2000-11-01,JAY,娘子,娘子 娘子却依旧每日 折一枝杨柳 你在那里 在小村外的溪边河口 默默等著我 娘子依旧每日折一...,方文山,娘子 默默
4,2000-11-01,JAY,斗牛,有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底跩什么 懂不懂篮球 有种不要走 三对三斗...,方文山,停留 篮下
5,2000-11-01,JAY,黑色幽默,难过 是因为闷了很久 是因为想了太多 是心理起了作用 你说 苦笑常常陪着你 在一起有点勉强 ...,周杰伦,黑色幽默 不想
6,2000-11-01,JAY,伊斯坦堡,走过了很多地方 我来到伊斯坦堡 就像是童话故事 有教堂有城堡 每天忙碌地寻找 到底什么我想要...,徐若瑄,想要 每天
7,2000-11-01,JAY,印地安老斑鸠,沙漠之中怎么会有泥鳅 话说完飞过一只海鸥 大峡谷的风呼啸而过 是谁说没有 有一条热昏头的响尾...,方文山,斑鸠 印地安
8,2000-11-01,JAY,龙卷风,爱像一阵风 吹完它就走 这样的节奏 谁都无可奈何 没有你以後 我灵魂失控 黑云在降落 我被它...,徐若瑄,不能 不要
9,2000-11-01,JAY,反方向的钟,迷迷蒙蒙 你给的梦 出现裂缝 隐隐作痛 怎么沟通 你都没空 说我不懂 说了没用 他的笑容 有...,方文山,没有 放纵


# flatten技巧

## itertools.chain

In [14]:
import itertools
list2d = [[1,2,3],[4,5,6], [7], [8,9]]
merged = list(itertools.chain(*list2d))
merged

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
pd.DataFrame(merged)

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9


## sum(x,[])

In [16]:
sum(list2d,[])

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [22]:
x = np.array([[1,2,3],[4,5,6]])
x

array([[1, 2, 3],
       [4, 5, 6]])

## np.ravel

In [23]:
np.ravel(x)

array([1, 2, 3, 4, 5, 6])

## merge

In [30]:
df = DataFrame({'A': [1, 2], 'B': [['a', 'b'], 'c']})
df

,A,B
0,1,"[a, b]"
1,2,c


In [31]:
b_flat = pd.DataFrame([[i, x] 
...               for i, y in df['B'].apply(list).iteritems() 
...                    for x in y], columns=list('IB'))

In [32]:
b_flat

,I,B
0,0,a
1,0,b
2,1,c


In [33]:
b_flat = b_flat.set_index('I')

In [34]:
b_flat

,B
I,
0,a
0,b
1,c


In [35]:
 df[['A']].merge(b_flat, left_index=True, right_index=True)

,A,B
0,1,a
0,1,b
1,2,c
